<B><h1><font color = "green" size = '20'>HANDOUT ASSIGNMENTS : </font></h1></B>

<h1><B><font color = 'red'>GPU OPTIMISATION</font><B></h1>

# In the below question, we want you to make some necessary changes, so that this code not only gives the output but also logs the device placement informations. 

# You can run the cell to see whether you succeeded or not.



In [36]:
## we are using tensorflow version 1, so , the below two lines of code is to 
# depriciate it from version 2 to version 1
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Construct 2 op nodes (m1, m2) representing 2 matrix.

m1 = tf.constant([[3, 5]])
m2 = tf.constant([[2],[4]])

product = tf.matmul(m1, m2)    # A matrix multiplication op node

sess = tf.Session(config=tf.ConfigProto(
    
    log_device_placement=True))
print(sess.run(product))

sess.close()

# MatMul: (MatMul): /job:localhost/replica:0/task:0/cpu:0
# Const_1: (Const): /job:localhost/replica:0/task:0/cpu:0
# Const: (Const): /job:localhost/replica:0/task:0/cpu:0

# In the next task, we are providing you a boiler plate code based on Model Parallelism. 
You have to complete the following task in your code:
We have already initialised two matrix of dimension 2X2, a and b, with random values.
Now you have to implement this three steps.


1.   Use gpu with id '0' to perform matrix multiplication and append it in  the list 'c'.
2.  Use gpu with id '1' to perform atrix multiplication of 'a' and 'b'. And append it in list 'c'.
3.  Add all the matrix which you appended in list 'c' above. You have to use cpu with id '0' for this step.



In [35]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# list 'c' in which we will append our list
c = []
# random initialisation of two matrices 'a' and 'b'.
a = tf.get_variable(f"a", [2, 2], initializer=tf.random_uniform_initializer(-1, 1))
b = tf.get_variable(f"b", [2, 2], initializer=tf.random_uniform_initializer(-1, 1))

##### START CODE HERE  #######

'''
HINTS :
You have to initialise three devices as asked above (gpu-0, gpu-1, cpu-0).
After initialisation perform the task in each f those initialisation.
For matrix multiplication, look at above task.
While for addition part - Find Yourself.
'''

#### YOU CODE ENDS HERE   ######

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True))

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run())

# Before moving to the next concept, We want you to have a quick revision of the difference between the Data Parallelism and Model Prarallelism. So, the below shown piece of code is based on Data Parallelism.
# The code will show an error, when you run it. You have to find out the bug and fix it (as simple as that).

In [ ]:

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

c = []
a = tf.get_variable(f"a", [2, 4, 3], initializer=tf.random_uniform_initializer(-1, 1))  # [2,2,3]
b = tf.get_variable(f"b", [2, 3, 2], initializer=tf.random_uniform_initializer(-1, 1))

# Multiple towers
for i, d in enumerate(['/gpu:0', '/gpu:1']):   # using gpu-0 and gpu-1 to preform matrix multiplication.
    with tf.device(d):
        c.append(tf.matmul(a[i], b[i]))   # Tower i is responsible for batch data i.

with tf.device('/cpu:0'):
    sum = tf.add_n(c)

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True))

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run())   #sum in the argument.



---

# <font color = "red">PANDARALLEL</font>
In this part we practice with the pandarallel library that allows parallel processing of DataFrames. 

#### Note: pandarallel can speed up computation using the physical cores present, hyperthreading does not increase speed. So it is recommended to run this in a multicore environment

## Setup
Run this code before attempting the excerise

In [ ]:
import numpy as np
import pandas as pd

size = int(5e6)

# Synthetic dataset of random points
df = pd.DataFrame({
    'XCoordinate': np.random.randn(size),
    'YCoordinate': np.random.randn(size),
    'Color': np.random.choice(['Red','Blue','Green'],size)
})
# Preview of dataset
df.head()

## Step 1
For starters you have to first import the `pandarallel` class from the pandarallel module and run the `initialize()` method of the class. (You may have to pip install the `pandarallel` module)


In [ ]:
# YOUR CODE HERE
from pandarallel import pandarallel # Example solution remove before assigning
pandarallel.initialize()            # ''

## Step 2
We have defined the function `l1_norm` here that takes a row in the `DataFrame` as argument and returns the L1 Norm of the point, i.e sum of absolute values of the coordinates. Run the cell.

In [ ]:
def l1_norm(row):
  return row.XCoordinate + row.YCoordinate

Next we will apply the function to each row by calling the `apply()` method on the `DataFrame`. This may take a minute or two. (If it takes too long you can change the `size` variable to a smaller number so that it runs for a minute or two)

In [ ]:
%%time
df.apply(l1_norm,axis=1)

Now your task is to process each row with the same function but now 

1.   List item
2.   List item

using parallel processing provided with the help of `pandarallel` . You can compre the time with the previous execution.

In [ ]:
%%time
df.parallel_apply(l1_norm,axis=1)  # Remove before assigning

Now hopefully if you had multiple cores you can appreciate the 

*   List item
*   List item

reduction in time offered by the parallel processing

## Step 3
In this step what you have to is to use what you have learned and add a column `ORD_Color` to the dataset containing the ordinal encoding of the `'Color'` column, you must use parallel processing.

In [ ]:
# YOUR CODE HERE
# Example solution remove before assigning
ord_map = {}
index = 0
for i in df.Color.unique():
    ord_map[i] = index
    index +=1

def encoder(row):
    return ord_map[row.Color]

df['ORD_Color'] = df.parallel_apply(encoder,axis=1)
df.head()

<font color = "Red"><B><h1>Mult Threading and Multi Processing</h1></B></font>

Before moving further, run the cell below to have an basic idea of Threading in Python.<br>
Here, in this code we have made three separate threads - 
1. The main Thread
2. t1 thread
3. t2 thread

t1 thread is responsible for printing 'Hello' while the t2 thread is responsible for 'Hi' and the main thread is the main thread, here in this case it is reponsible for printing 'Bye'.

In [ ]:
## The first imports the sleep function from time module- it is used wait at the line for a given time.
## The second line is to import the inbuilt 'Threading' module of Python.


from time import sleep 
from threading import *

class Hello(Thread):
  def run(self):  
    for i in range(5):
      print("Hello")
      sleep(1)  # waits for 1 milli seconds at this line.

class Hi(Thread):
  def run(self):
    for i in range(5):
      print("Hi")
      sleep(1)

t1 = Hello()
t2 = Hi()

t1.start()   # calls the run function of t1 class (Hello class). 
sleep(0.2)
t2.start()   # calss the run function of the t2 class (Hi class).

'''
The below lines are helpful in executing the t1 thread and t2 thread first before the 
execution of Main Thread. To understand more clearly, just comment out the two lines 
and then run the cell to see the difference.
'''
t1.join()
t2.join()

print("Bye")


'''
If you want to explre more: We have a small piece of code for your help:
Just uncomment it and run it in a different cell.
'''
#: Hope you remember the Process function from the slides !

# from multiprocessing import Process   

# def f(name):
#     print('hello', name)

# if __name__ == '__main__':
#     p = Process(target=f, args=('bob',))
#     p.start()
#     p.join()

'''
Bonus Task:
With the help of multiprocessing module of python, try to initialise the 't1' and 
't2' as Process() [ as did in commented code ], and so try to improve the functionality of this code.
Watch this video to get more idea -
'https://www.youtube.com/watch?v=fKl2JW_qrso'

'''
  

# In this task you have to literally do nothing !
# You just have to twik the range of the iteration(the for loop) to figure out the relationship between the overall time taken to perform the task and the computation complexity of the task

In [ ]:
'''
INSTRUCTIONS BEFORE YOU RUN THIS CELL:
We highly recommend to put this code in your system itself.
You can use either of .py file or .ipnyb file.
And then run the below code to get an intution of what is happenening, here.
'''

import multiprocessing
from multiprocessing import Pool
import time

def square_add(x):
  sum = 0
  for j in range(1000):
    sum += j*j
  return sum

if __name__ == "__main__":
      
      toc = time.time()
      p = Pool()
      result = p.map(square_add, range(10000))
      p.close()
      p.join()
      tic = time.time()
      print("Pool time = ", tic-toc)

      toc = time.time()
      result = []
      
      for i in range(10000):
          result.append(square_add(i))
      tic = time.time()
      print("Serial Processing time = ", tic-toc)


# The Task below is designed to get your hands on the type of Shared Memory among processes as discussed in the slides.<br>
Read the INSTRUCTIONS carefully to get an idea of your task.

In [ ]:
'''
INSTRUCTIONS:
1. FIRST of all COMMENT the four line of codes (as instructed).
2. Once done that run the cell to see the output. Now, UNCOMMENT the said four lines.
3. Now, run the cell and see the output.
4. You might have noticed the difference in output-in one case ou have blank array while you got filled array in other case.
5. The reason behind this was - those four lines just make the memory sharing possible between the two processes.
6. One in he main process (under the if __name__ == "__main__: command) and the other is in the 'getSquare' function.
7. This is called 'queue pipe Multiprocessing'.

YOUR TASK:
1. Create a function named 'getNumSquare'. In this function you will take one argument 'num'
2. This 'num' is a shared Memory of type 'value' and is to be declared as 'double'.
3. You just have to assign a real number to the 'num' variable inside the function and get it printed 
  in the 'if statement' (if __name_ == "__main__")
4. We have implemented multiprocessing.active_children() for you in the bottom of code
5. You job is to just add 1 line of code to get the numbers of CPU in the system.(There is a inbulit function for this).

BONUS TASK:
1. Repeat the same task for a Queue type shared Memory.
2. You have to use queue.put()- to put the numbers in a variable named 'queue' of type Queue.
3. Also queue.get() - to fetch the data in the queue.
''' 

import multiprocessing

def getSquare(numbers, result):    
  for index, num in enumerate(numbers):
    result[index] = num*num

if __name__ == "__main__":
  numbers = [2,3,5]
  result = []

  ###  COMMENT FOUR LINES BELOW  ####
  result = multiprocessing.Array('i', 3)
  p = multiprocessing.Process(target=getSquare, args=(numbers, result))

  p.start()
  p.join()

  #### THE FOUR LINES TO BE COMMENTED   #######

  print(multiprocessing.active_children())
  print(result[:])